In [3]:
import json
from tqdm import tqdm
import pandas as pd
import re
import numpy as np

In [4]:
# Load intersection

In [5]:
path = "data/dataframes/full_pubs.pkl"
df = pd.read_pickle(path)


In [6]:
inter = df[df['Intersection']]
inter

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,SDG15,SDG16,SDG17,additive_manufacturing,AI,big_data,blockchain,computing_infrastructure,IOT,robotics
6,J,"Kao, CH; Wang, CC; Wang, HY","Kao, Chih-Han; Wang, Chien-Chih; Wang, Her -Yung",A neural-based predictive model of the compres...,COMPUTERS AND CONCRETE,Article,waste LCD concrete; compressive strength; back...,MECHANICAL-PROPERTIES; REGRESSION,The Taiwanese liquid crystal display (LCD) ind...,"[Kao, Chih-Han] Natl Quemoy Univ, Dept Civil E...",...,True,False,False,False,True,False,False,False,False,False
28,C,"Nugra, H; Abad, A; Fuertes, W; Galarraga, F; A...","Nugra, Hugo; Abad, Alejandra; Fuertes, Walter;...",A Low-cost IoT Application for the Urban Traff...,2016 IEEE/ACM 20TH INTERNATIONAL SYMPOSIUM ON ...,Proceedings Paper,Congestion of vehicular traffic; Wireless Sens...,INTERNET; SYSTEM; THINGS,Congestion of vehicular traffic in the cities ...,"[Nugra, Hugo; Abad, Alejandra; Fuertes, Walter...",...,False,False,False,False,True,False,False,False,False,False
90,J,"Peng, YQ; Xiao, YX; Fu, ZT; Dong, YH; Zheng, Y...","Peng, Yaoqi; Xiao, Yingxin; Fu, Zetian; Dong, ...",Precision irrigation perspectives on the susta...,JOURNAL OF CLEANER PRODUCTION,Article,Field crop; Water demand prediction; Irrigatio...,REGRESSION-MODEL; DRIP IRRIGATION; NEURAL-NETW...,There is a global shortage of fresh water to m...,"[Peng, Yaoqi; Xiao, Yingxin; Fu, Zetian; Dong,...",...,False,False,False,False,True,False,False,False,False,False
101,J,"Alvarez-de-los-Mozos, E; Renteria-Bilbao, A; D...","Alvarez-de-los-Mozos, Esther; Renteria-Bilbao,...",WEEE Recycling and Circular Economy Assisted b...,APPLIED SCIENCES-BASEL,Article,human-robot collaboration; collaborative robot...,ELECTRONIC EQUIPMENT WEEE,Considering the amount of waste of electrical ...,"[Alvarez-de-los-Mozos, Esther; Diaz-Martin, Fe...",...,False,False,False,False,False,False,False,False,False,True
218,J,"Laskurain-Iturbe, I; Arana-Landin, G; Landeta-...","Laskurain-Iturbe, Iker; Arana-Landin, German; ...",Exploring the influence of industry 4.0 techno...,JOURNAL OF CLEANER PRODUCTION,Article,Industry 4; 0; Circular economy; Literature re...,SMART PRODUCTION; SUSTAINABILITY; CHALLENGES; ...,"In the last decade, both Industry 4.0 technolo...","[Laskurain-Iturbe, Iker; Arana-Landin, German]...",...,False,False,False,True,True,True,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244072,J,"Strengers, Y; Nicholls, L","Strengers, Yolande; Nicholls, Larissa",Aesthetic pleasures and gendered tech-work in ...,MEDIA INTERNATIONAL AUSTRALIA,Article,aesthetics; digital housekeeping; gender; hous...,NaN,This article analyses visions of everyday life...,"[Strengers, Yolande; Nicholls, Larissa] RMIT U...",...,False,False,False,False,False,False,False,False,True,False
1244214,J,"Kan, MY; Sullivan, O; Gershuny, J","Kan, Man Yee; Sullivan, Oriel; Gershuny, Jonathan",Gender Convergence in Domestic Work: Discernin...,SOCIOLOGY-THE JOURNAL OF THE BRITISH SOCIOLOGI...,Article,domestic work; gender convergence; time use,WELFARE-STATE; TIME-USE; CONTEXT; EQUITY,Cross-national trends in paid and unpaid work ...,"[Kan, Man Yee; Sullivan, Oriel] Univ Oxford, D...",...,False,False,False,False,False,True,False,False,False,False
1244296,S,"Cornwell, B; Gershuny, J; Sullivan, O","Cornwell, Benjamin; Gershuny, Jonathan; Sulliv...",The Social Structure of Time: Emerging Trends ...,"ANNUAL REVIEW OF SOCIOLOGY, VOL 45",Article; Book Chapter,time use; time allocation; gender division of ...,PARENTS INVESTING LESS; DIVISION-OF-LABOR; CHI...,Research on time use has seen several major de...,"[Cornwell, Benjamin] Cornell Univ, Dept Sociol...",...,False,False,False,False,False,False,False,False,True,False
1244855,J,"Abdulcadir, J; Dewaele, R; Firmenich, N; Remui...","Abdulcadir, Jasmine; Dewaele, Romain; Firmenic...",In Vivo Imaging-Based 3-Dimensional Pelvic Pro...,JOURNAL OF SEXUAL MEDICINE,Article,Clitoris; Penis; Female

In [7]:
import json
df_query = pd.read_pickle("/home/kevin-desktop/PycharmProjects/SDG/query/query.pkl")


def to_raw(string):
    return fr"{string.strip()}"


def check_for_sdg(txt, how='dic'):
    if how == "dic":
        res = {"SDG"+str(i): [] for i in range(1,18)}
        for sdg, target, query, not_query in df_query.itertuples(index=False, name=None):
            if not not_query:
                try:
                    m = re.search(to_raw(query), txt, re.IGNORECASE)
                    if m:
                        # res.append((sdg, target, m.group()))
                        res[sdg].append((target, m.group()))
                except:
                    print("error, weird query")
            else:
                m = re.search(to_raw(query), txt, re.IGNORECASE)
                m_not = re.search(to_raw(not_query), txt, re.IGNORECASE)
                if m and not m_not:
                    # res.append((sdg, target, m.group()))
                    res[sdg].append((target, m.group()))
        if how == "dic":
            return {sdg:lst for sdg,lst in res.items() if lst}
    if how == "keywords":
        res = []
        for sdg, target, query, not_query in df_query.itertuples(index=False, name=None):
            if not not_query:
                try:
                    m = re.search(to_raw(query), txt, re.IGNORECASE)
                    if m:
                        # res.append((sdg, target, m.group()))
                        res.append(m.group())
                except:
                    print("error, weird query")
            else:
                m = re.search(to_raw(query), txt, re.IGNORECASE)
                m_not = re.search(to_raw(not_query), txt, re.IGNORECASE)
                if m and not m_not:
                    # res.append((sdg, target, m.group()))
                    res.append(m.group())
        return ", ".join(res)
    else:
        print("wrong argument")
    # return json.dumps({sdg:lst for sdg,lst in res.items() if lst}, indent=4)

In [8]:
check_for_sdg(inter.AB.values[0], how="keywords")

'Waste glass recycling can reduce , Waste glass recycling can reduce,  placed in landfills. Waste glass recycling can reduce, sustainable environmental ,  concrete and promote sustainable environmental'

In [9]:
df_digi_keywords = pd.read_csv("data/keyword_digital_simplified.csv").transpose()
dic_digi = {}
set_cat = set()
for digi_cat, *digi_keywords in df_digi_keywords.itertuples(name=None):
    set_cat.add(digi_cat)
    for keyword in digi_keywords:
        try:
            dic_digi[keyword.replace("*", "\w*")] = digi_cat
        except:
            pass
def check_for_dt(txt:str, how="dic"):
    if how=="dic":
        res = {cat:[] for cat in set_cat}
        for key, cat in dic_digi.items():
             key = to_raw(key.replace(" ", "[\s-]"))
             m = re.search(key, txt, re.IGNORECASE)
             if m:
                 res[cat].append(m.group())
        return {dt:lst for dt,lst in res.items() if lst}
    elif how == "keywords":
        res = []
        for key, cat in dic_digi.items():
             key = to_raw(key.replace(" ", "[\s-]"))
             m = re.search(key, txt, re.IGNORECASE)
             if m:
                 res.append(m.group())
        return ", ".join(res)
    else:
        print("wrong argument")


In [10]:
check_for_dt(inter.AB.values[0], how="keywords")

'Neural networks, supervised learning'

In [11]:
cond = inter['SDG3'] | inter['SDG5'] | inter['SDG7'] | inter['SDG10'] | inter['SDG11'] | inter['SDG13']
smaller_inter = inter[cond]

In [12]:
def target_to_sdg(s:set):
    new_s = set()
    accepted_vals = [3,5,7,8,10,11,13]
    for elem in s:
        new_val = int(elem.split(".")[0])
        if new_val in accepted_vals:
            new_s.add(new_val)

    return new_s.pop()


In [13]:
import numpy as np

smaller_inter.loc[:, 'SDG'] = np.vectorize(target_to_sdg)(smaller_inter.loc[:, 'lst_Target'])


/tmp/ipykernel_33473/2218458147.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller_inter.loc[:, 'SDG'] = np.vectorize(target_to_sdg)(smaller_inter.loc[:, 'lst_Target'])


In [14]:
smaller_inter = smaller_inter.groupby('SDG').sample(10).reset_index(drop=True)
smaller_inter['TXT'] = smaller_inter['AB']+"    "+smaller_inter['TI']+"    "+smaller_inter['DE']+ "    " + smaller_inter['ID'].fillna("")


In [15]:
import swifter
tqdm.pandas()
smaller_inter['sdgkey'] = smaller_inter['TXT'].swifter.apply(lambda x: check_for_sdg(x, how="keywords"))

/home/kevin-desktop/anaconda3/envs/datascience/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Pandas Apply:   0%|          | 0/70 [00:00<?, ?it/s]

In [16]:
tqdm.pandas()
smaller_inter['dtkey'] = smaller_inter['TXT'].swifter.apply(lambda x: check_for_dt(x))

Pandas Apply:   0%|          | 0/70 [00:00<?, ?it/s]

In [17]:
smaller_inter

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,additive_manufacturing,AI,big_data,blockchain,computing_infrastructure,IOT,robotics,TXT,sdgkey,dtkey
0,J,"Froz, BR; de Carvalho, AO; Silva, AC; de Paiva...","Froz, Bruno Rodrigues; de Carvalho Filho, Anto...",Lung nodule classification using artificial cr...,EXPERT SYSTEMS WITH APPLICATIONS,Article,Artificial life; Artificial crawlers; Rose dia...,SEGMENTATION; AREA,Lung cancer is the major cause of death among ...,"[Froz, Bruno Rodrigues; de Carvalho Filho, Ant...",...,False,True,False,False,False,False,False,Lung cancer is the major cause of death among ...,"cancer has the highest mortality, death among ...",expert system
1,J,"Dash, M; Shadangi, PY; Muduli, K; Luhach, AK; ...","Dash, Manoranjan; Shadangi, Preeti Y.; Muduli,...",Predicting the motivators of telemedicine acce...,JOURNAL OF STATISTICS & MANAGEMENT SYSTEMS,Article,Telemedicine; TAM model; ANN; Multiple regress...,NaN,"Since the outbreak of COVID-19, healthcare pro...","[Dash, Manoranjan; Shadangi, Preeti Y.] Siksha...",...,False,True,False,False,False,False,False,"Since the outbreak of COVID-19, healthcare pro...","Pandemic; COVID-19, COVID-19 pandemic",Neural network
2,J,"Lin, CM; Hou, YL; Chen, TY; Chen, KH","Lin, Chih-Min; Hou, Yu-Ling; Chen, Te-Yu; Chen...",Breast Nodules Computer-Aided Diagnostic Syste...,IEEE TRANSACTIONS ON FUZZY SYSTEMS,Article,Breast nodule classification; cerebellar model...,CLASSIFICATION; ULTRASOUND; LESIONS,Since the mortality rate of breast cancer in w...,"[Lin, Chih-Min] Yuan Ze Univ, Dept Elect Engn,...",...,False,True,False,False,False,False,False,Since the mortality rate of breast cancer in w...,mortality rate of breast cancer,neural network
3,J,"Yousaf, I","Yousaf, Imran",Risk transmission from the COVID-19 to metals ...,RESOURCES POLICY,Article,The COVID-19; Precious metals; Industrial meta...,VOLATILITY SPILLOVERS; OIL PRICES; STOCK; LINK...,We examine the risk transmission from the COVI...,"[Yousaf, Imran] Air Univ, Sch Management, Isla...",...,False,False,False,True,False,False,False,We examine the risk transmission from the COVI...,COVID-19 pandemic,CRYPTOCURRENCIES
4,J,"Na, KJ; Park, S; Park, IK; Kim, YT; Kang, CH","Na, Kwon Joong; Park, Santinit; Park, In Kyu; ...",Outcomes after total robotic esophagectomy for...,JOURNAL OF THORACIC DISEASE,Article,Robotic surgery; esophageal surgery; minimally...,MINIMALLY INVASIVE ESOPHAGECTOMY; ASSISTED THO...,Background: Robot-assisted minimally invasive ...,"[Na, Kwon Joong; Park, Santinit; Park, In Kyu;...",...,False,False,False,False,False,False,True,Background: Robot-assisted minimally invasive ...,,"Robot, robotic"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,C,"Nimmagadda, SL; Dreher, H; Mora, PAC; Lobo, A","Nimmagadda, Shastri L.; Dreher, Heinz; Cardona...",Ontology based Multidimensional Data Warehousi...,2013 11TH IEEE INTERNATIONAL CONFERENCE ON IND...,Proceedings Paper,Data warehousing; multidimensional; ontology d...,NaN,A full understanding of many unconventional hy...,"[Nimmagadda, Shastri L.] Schlumberger, PTS, Mo...",...,False,True,False,False,False,False,False,A full understanding of many unconventional hy...,,data-mining
66,C,"Zhu, MH; Zhu, WC; Yu, XB; Yang, JL; Ding, DW",Zhu Minghe; Zhu Wencai; Yu Xubo; Yang Jianli; ...,Two Indirect Methods about Environmental Damag...,BEYOND EXPERIENCE IN RISK ANALYSIS AND CRISIS ...,Proceedings Paper,ship oil spill accident; environmental damage;...,NaN,Two methods based on GA-SVM[genetic algorithm ...,"[Zhu Minghe; Zhu Wencai; Yu Xubo] Ningbo Univ,...",...,False,True,False,False,False,False,False,Two methods based on GA-SVM[genetic algorithm ...,,Neural Network
67,J,"Ramos-Giraldo, P; Reberg-Horton, C; Locke, AM;...","Ramos-Giraldo, Paula; Reberg-Horton, Chris; Lo...",Drought Stress Detection Using Low-Cost Comput...,IT PROFESSIONAL,Article,Agriculture; Stress; Machine learning; Compute...,NaN,The real-time detection of drought stress has ...,"[Ra

In [18]:
smaller_inter = smaller_inter[['TI', 'DE', 'SDG', 'AB', 'AI', 'robotics', 'IOT', 'big_data', 'computing_infrastructure', 'additive_manufacturing', 'blockchain', 'sdgkey', 'dtkey']]

In [19]:
smaller_inter.to_pickle("data/dataframes/output/marco_sample.pkl")

In [37]:

smaller_inter = smaller_inter.replace("", np.NAN).dropna(subset='dtkey').groupby('SDG').sample(20).reset_index(drop=True)

In [39]:
smaller_inter.to_pickle(("data/dataframes/output/sample140.pkl"))

In [37]:
check_for_dt(smaller_inter.loc[15, "TXT"])

{'AI': ['NEURAL-NETWORKS']}